# Testing notebook

This notebook is used to produce the testing by the tools. 

The goal here is simply to call the APIs in question and record their answers on our datasets. No analysis is conducted here, this is purely to get their answers for later. 

Here, we test the following tools : 
- Namsor
- GenderAPI.io
- GenderAPI.com
- Genderize
- gender_guesser

The last one, gender_guesser, is not an API, so it is not tested along the rest of the other tools, but is done in its own cells. 

Once the results of all those tools are defined, the results are stored as `.csv` file for analysis later. 

In [1]:
import pandas as pd
%cd ../..
from notebooks.helpers.serviceWrapperAsync import NamSorEndpoint 
from notebooks.helpers.serviceHandlers import GenderGuesserHandler
from notebooks.helpers.wrapperManager import WrapperManager
import notebooks.helpers.serviceWrapperAsync as wrapper
from notebooks.helpers.run_batched import run_batched
import asyncio
import random
%load_ext autoreload
%autoreload 2

C:\Users\brief\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche


In [2]:
master_df = pd.read_csv('C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/all.csv')

In [3]:
master_df.head()

,id,fullName,firstName,lastName,gender,isoCountry,continent,birthYear,hasMiddleName,hasNoLastName,source
0,0,Clara Benson,Clara,Benson,female,GH,Africa,2000,False,False,wikidata
1,1,Esther Ruth Mbabazi,NaN,NaN,female,UG,Africa,1995,True,False,wikidata
2,2,Dalia Ziada,Dalia,Ziada,female,EG,Africa,1982,False,False,wikidata
3,3,Fatou Haidara,Fatou,Haidara,female,ML,Africa,1962,False,False,wikidata
4,4,Claude Haffner,Claude,Haffner,female,NaN,NaN,1976,False,False,wikidata


## Wikidata first name + localization

In [4]:
wikidata_First_Country = master_df[
    (master_df['source'] == 'wikidata') &
    (master_df['firstName'].notnull()) &
    (master_df['isoCountry'].notnull())
]
wikidata_First_Country.shape

(3645, 11)

### Genderize

In [132]:
result_Wikidata_First_Country_Genderize = await wrapper.GenderizeWrapper(wikidata_First_Country).get_prediction_async(useLocalization=True)
result_Wikidata_First_Country_Genderize

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,0,Clara Benson,Clara,female,female,GH,True,genderize.IO,247083,0.99
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderize.IO,87214,0.99
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderize.IO,25477,0.98
3,7,Diallo Sène,Diallo,female,male,ML,True,genderize.IO,8032,0.82
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderize.IO,15498,0.99
...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderize.IO,324,1.00
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderize.IO,3124,0.96
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderize.IO,478589,1.00
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderize.IO,3,1.00


In [139]:
result_Wikidata_First_Country_Genderize.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Genderize/wikidataFirstCountry.csv", index=False)

### GenderAPI.io

In [20]:
result_Wikidata_First_Country_GenderAPIIO = await run_batched(
    wrapper_class=wrapper.GenderAPI_IO_Wrapper,
    df=wikidata_First_Country,
    batch_size=200,
    pause_seconds=2,
    useLocalization=True
)

Output()

[17:26:34] Processing rows 0-200...                                                               ]8;id=422503;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=770372;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[17:26:35] Cooling down for 5.8s to respect API limits...                                         ]8;id=837113;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=753544;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[17:26:43] Cooling down for 4.3s to respect API limits...                                         ]8;id=116746;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=107045;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[17:26:47] Processing rows 400-600...                                                             ]8;id=389047;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=915719;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

...

[17:28:25] Processing rows 3200-3400...                                                           ]8;id=162160;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=370334;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[17:28:29] Cooling down for 6.6s to respect API limits...                                         ]8;id=167641;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=149918;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[17:28:35] Processing rows 3400-3600...                                                           ]8;id=234887;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=170600;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[17:28:37] Cooling down for 5.0s to respect API limits...                                         ]8;id=19753;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=300627;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[17:28:42] Processing rows 3600-3645...                                                           ]8;id=602459;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=132250;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

In [23]:
result_Wikidata_First_Country_GenderAPIIO['predictedGender'].value_counts(dropna=False)

predictedGender
male      1795
female    1732
null       118
Name: count, dtype: int64

In [24]:
result_Wikidata_First_Country_GenderAPIIO.loc[
    result_Wikidata_First_Country_GenderAPIIO['predictedGender']=='null', 'predictedGender'] = 'unknown'
result_Wikidata_First_Country_GenderAPIIO['predictedGender'].value_counts(dropna=False)

predictedGender
male       1795
female     1732
unknown     118
Name: count, dtype: int64

In [25]:
result_Wikidata_First_Country_GenderAPIIO['correctGender'].value_counts(dropna=False)

correctGender
female    1875
male      1770
Name: count, dtype: int64

In [26]:
result_Wikidata_First_Country_GenderAPIIO.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIIO/wikidataFirstCountry.csv", index=False)

### GenderAPI.com

In [ ]:
result_Wikidata_First_Country_GenderAPIcom = await run_batched(
    wrapper.GenderAPI_com_Wrapper,
    wikidata_First_Country,
    batch_size=50,
    pause_seconds=10,
    useLocalization=True,
    useFullName=False
)
result_Wikidata_First_Country_GenderAPIcom


Processing rows 0-50...
Sleeping 12.7s to respect API limits...

Processing rows 50-100...
Sleeping 11.5s to respect API limits...

Processing rows 100-150...
Sleeping 13.3s to respect API limits...

Processing rows 150-200...
Sleeping 11.1s to respect API limits...

Processing rows 200-250...
Sleeping 14.3s to respect API limits...

Processing rows 250-300...
Sleeping 13.4s to respect API limits...

Processing rows 300-350...
Sleeping 12.7s to respect API limits...

Processing rows 350-400...
Sleeping 11.3s to respect API limits...

Processing rows 400-450...
Sleeping 12.6s to respect API limits...

Processing rows 450-500...
Sleeping 14.7s to respect API limits...

Processing rows 500-550...
Sleeping 14.0s to respect API limits...

Processing rows 550-600...
Sleeping 13.6s to respect API limits...

Processing rows 600-650...
Sleeping 14.5s to respect API limits...

Processing rows 650-700...
Sleeping 13.2s to respect API limits...

Processing rows 700-750...
Sleeping 14.7s to respec

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,0,Clara Benson,Clara,female,female,GH,True,genderAPI.com,False,0.98,Clara,None
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderAPI.com,False,0.98,Dalia,None
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderAPI.com,False,0.97,Fatou,None
3,7,Diallo Sène,Diallo,female,male,ML,True,genderAPI.com,False,0.86,Diallo,None
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderAPI.com,False,0.98,Fatoumata,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderAPI.com,False,0.99,Ícaro,None
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderAPI.com,False,1.0,Sir,None
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderAPI.com,False,1.0,Julio,None
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderAPI.com,False,0.75,Wrays,None


In [23]:
result_Wikidata_First_Country_GenderAPIcom['predictedGender'].value_counts()

predictedGender
male       1781
female     1734
unknown      24
ERROR        18
Name: count, dtype: int64

An error appeared when testing the people who comes from South Sudan. Their ISO code (SS) was not listed within GenderAPI.com, which triggered an error. 

For the moment, for ease of use, I take the decision to keep those listing as is, with the ERROR code for some elements. At a later date, those rows may be either retried with a different ISO code (for example, SD, for Sudan) or may be deleted. 

In [33]:
result_Wikidata_First_Country_GenderAPIcom.sort_values(by=['index'])

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,0,Clara Benson,Clara,female,female,GH,True,genderAPI.com,False,0.98,Clara,None
1,2,Dalia Ziada,Dalia,female,female,EG,True,genderAPI.com,False,0.98,Dalia,None
2,3,Fatou Haidara,Fatou,female,female,ML,True,genderAPI.com,False,0.97,Fatou,None
3,7,Diallo Sène,Diallo,female,male,ML,True,genderAPI.com,False,0.86,Diallo,None
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderAPI.com,False,0.98,Fatoumata,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,genderAPI.com,False,0.99,Ícaro,None
3641,4865,Sir Chandler,Sir,male,male,AR,True,genderAPI.com,False,1.0,Sir,None
3642,4866,Julio Fierro,Julio,male,male,CL,True,genderAPI.com,False,1.0,Julio,None
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,genderAPI.com,False,0.75,Wrays,None


In [34]:
result_Wikidata_First_Country_GenderAPIcom.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIcom/wikidataFirstCountry.csv", index=False)

### Namsor

In [ ]:
result_Wikidata_First_Country_Namsor = await run_batched(
    wrapper.NamSorWrapper,
    wikidata_First_Country,
    batch_size=50,
    pause_seconds=10,
    endpoint=NamSorEndpoint.FIRST_NAME_GEO
)
result_Wikidata_First_Country_Namsor


Processing rows 0-50...
Sleeping 13.8s to respect API limits...

Processing rows 50-100...
Sleeping 13.2s to respect API limits...

Processing rows 100-150...
Sleeping 14.5s to respect API limits...

Processing rows 150-200...
Sleeping 14.1s to respect API limits...

Processing rows 200-250...
Sleeping 12.8s to respect API limits...

Processing rows 250-300...
Sleeping 12.5s to respect API limits...

Processing rows 300-350...
Sleeping 13.2s to respect API limits...

Processing rows 350-400...
Sleeping 11.4s to respect API limits...

Processing rows 400-450...
Sleeping 11.9s to respect API limits...

Processing rows 450-500...
Sleeping 13.8s to respect API limits...

Processing rows 500-550...
Sleeping 11.5s to respect API limits...

Processing rows 550-600...
Sleeping 13.3s to respect API limits...

Processing rows 600-650...
Sleeping 12.9s to respect API limits...

Processing rows 650-700...
Sleeping 13.0s to respect API limits...

Processing rows 700-750...
Sleeping 13.3s to respec

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraGenderScale,extraScore,extraProbabilityCalibrated,extraScript
0,0,Clara Benson,Clara,female,female,GH,True,NamSor,0.999099,15.669216,0.999549,LATIN
1,2,Dalia Ziada,Dalia,female,female,EG,True,NamSor,0.998789,15.373957,0.999395,LATIN
2,3,Fatou Haidara,Fatou,female,female,ML,True,NamSor,0.987619,12.789984,0.993810,LATIN
3,7,Diallo Sène,Diallo,female,female,ML,True,NamSor,0.310045,1.876022,0.655023,LATIN
4,8,Fatoumata Sylla,Fatoumata,female,female,ML,True,NamSor,0.999991,20.272138,0.999995,LATIN
...,...,...,...,...,...,...,...,...,...,...,...,...
3640,4864,Ícaro Lira,Ícaro,male,male,BR,True,NamSor,-0.997736,14.748236,0.998868,LATIN
3641,4865,Sir Chandler,Sir,male,male,AR,True,NamSor,-0.964407,9.258472,0.982204,LATIN
3642,4866,Julio Fierro,Julio,male,male,CL,True,NamSor,-0.993507,13.694498,0.996753,LATIN
3643,4870,Wrays Pérez,Wrays,male,male,PE,True,NamSor,-0.228425,1.480006,0.614213,LATIN


In [12]:
result_Wikidata_First_Country_Namsor['predictedGender'].value_counts()

predictedGender
male      1825
female    1820
Name: count, dtype: int64

In [17]:
result_Wikidata_First_Country_Namsor.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Namsor/wikidataFirstCountry.csv", index=False)

### GenderGuesser

In [30]:
genderGuesserHandler = GenderGuesserHandler(wikidata_First_Country)

result_Wikidata_First_GenderGuesser = genderGuesserHandler.get_prediction(useLocalization=True)

In [31]:
result_Wikidata_First_GenderGuesser['extraPreciseGenderPredicted'].value_counts()

extraPreciseGenderPredicted
andy             1560
unknown           821
male              611
female            565
mostly_male        45
mostly_female      43
Name: count, dtype: int64

In [32]:
result_Wikidata_First_GenderGuesser.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderGuesser/wikidataFirstCountry.csv", index=False)

## Kaggle First + Localization

In [3]:
kaggle_First_Country = master_df[
    (master_df['source'] == 'kaggleOlympic') &
    (master_df['firstName'].notnull()) &
    (master_df['isoCountry'].notnull())
]
kaggle_First_Country.shape

(9078, 11)

### Genderize

In [ ]:
result_Kaggle_First_Country_Genderize = await run_batched(
    wrapper_class=wrapper.GenderizeWrapper,
    df=kaggle_First_Country,
    batch_size=100,
    pause_seconds=5,
    useLocalization=True
)
result_Kaggle_First_Country_Genderize


Processing rows 0-100...
Sleeping 6.2s to respect API limits...

Processing rows 100-200...
Sleeping 6.7s to respect API limits...

Processing rows 200-300...
Sleeping 8.5s to respect API limits...

Processing rows 300-400...
Sleeping 7.3s to respect API limits...

Processing rows 400-500...
Sleeping 7.8s to respect API limits...

Processing rows 500-600...
Sleeping 9.6s to respect API limits...

Processing rows 600-700...
Sleeping 9.5s to respect API limits...

Processing rows 700-800...
Sleeping 7.5s to respect API limits...

Processing rows 800-900...
Sleeping 7.9s to respect API limits...

Processing rows 900-1000...
Sleeping 7.8s to respect API limits...

Processing rows 1000-1100...
Sleeping 8.4s to respect API limits...

Processing rows 1100-1200...
Sleeping 7.1s to respect API limits...

Processing rows 1200-1300...
Sleeping 8.0s to respect API limits...

Processing rows 1300-1400...
Sleeping 6.3s to respect API limits...

Processing rows 1400-1500...
Sleeping 9.3s to respect 

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,4874,ALEKSANYAN Artur,Artur,male,male,AM,True,genderize.IO,138242,1.00
1,4875,AMOYAN Malkhas,Malkhas,male,male,AM,True,genderize.IO,3,1.00
2,4876,GALSTYAN Slavik,Slavik,male,male,AM,True,genderize.IO,2018,0.99
3,4877,HARUTYUNYAN Arsen,Arsen,male,male,AM,True,genderize.IO,10342,0.99
4,4878,TEVANYAN Vazgen,Vazgen,male,male,AM,True,genderize.IO,398,0.98
...,...,...,...,...,...,...,...,...,...,...
9073,15980,HOMAN Khrystyna,Khrystyna,female,female,UA,True,genderize.IO,2032,1.00
9074,15983,LIUZZI Emanuela,Emanuela,female,female,IT,True,genderize.IO,91696,1.00
9075,15984,BOERS Isayah,Isayah,male,male,NL,True,genderize.IO,86,0.97
9076,15985,STAUT Kevin,Kevin,male,male,FR,True,genderize.IO,1257153,1.00


In [37]:
result_Kaggle_First_Country_Genderize.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Genderize/kaggleFirstCountry.csv", index=False)

### GenderAPI.io

In [ ]:
result_Kaggle_First_Country_GenderAPIIO = await run_batched(
    wrapper_class=wrapper.GenderAPI_IO_Wrapper,
    df=kaggle_First_Country,
    batch_size=200,
    pause_seconds=5,
    useLocalization=True
)
result_Kaggle_First_Country_GenderAPIIO


Processing rows 0-200...
Sleeping 8.6s to respect API limits...

Processing rows 200-400...
Sleeping 7.7s to respect API limits...

Processing rows 400-600...
Sleeping 9.8s to respect API limits...

Processing rows 600-800...
Sleeping 6.3s to respect API limits...

Processing rows 800-1000...
Sleeping 6.4s to respect API limits...

Processing rows 1000-1200...
Sleeping 6.1s to respect API limits...

Processing rows 1200-1400...
Sleeping 8.4s to respect API limits...

Processing rows 1400-1600...
Sleeping 10.0s to respect API limits...

Processing rows 1600-1800...
Sleeping 8.9s to respect API limits...

Processing rows 1800-2000...
Sleeping 7.4s to respect API limits...

Processing rows 2000-2200...
Sleeping 6.0s to respect API limits...

Processing rows 2200-2400...
Sleeping 6.5s to respect API limits...

Processing rows 2400-2600...
Sleeping 8.8s to respect API limits...

Processing rows 2600-2800...
Sleeping 8.0s to respect API limits...

Processing rows 2800-3000...
Sleeping 6.9s 

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraTotalName,extraProbability,extraCountryUsedByService,extraUsedAI,extraForcedGenderize
0,4874,ALEKSANYAN Artur,artur,male,male,AM,True,genderAPI.io,100,100,AM,False,False
1,4875,AMOYAN Malkhas,,male,null,AM,True,genderAPI.io,1,50,AM,False,False
2,4876,GALSTYAN Slavik,slavik,male,male,AM,True,genderAPI.io,346,100,UA,False,False
3,4877,HARUTYUNYAN Arsen,arsen,male,male,AM,True,genderAPI.io,99,99,AM,False,False
4,4878,TEVANYAN Vazgen,vazgen,male,male,AM,True,genderAPI.io,17,100,AM,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9073,15980,HOMAN Khrystyna,khrystyna,female,female,UA,True,genderAPI.io,168,100,UA,False,False
9074,15983,LIUZZI Emanuela,emanuela,female,female,IT,True,genderAPI.io,788,100,IT,False,False
9075,15984,BOERS Isayah,isayah,male,male,NL,True,genderAPI.io,6,100,US,False,False
9076,15985,STAUT Kevin,kevin,male,male,FR,True,genderAPI.io,723,99,FR,False,False


In [9]:
result_Kaggle_First_Country_GenderAPIIO['predictedGender'].value_counts()

predictedGender
male       4730
female     4097
unknown     250
              1
Name: count, dtype: int64

In [15]:
acceptedGender = ['male', 'female', 'unknown']

result_Kaggle_First_Country_GenderAPIIO[~result_Kaggle_First_Country_GenderAPIIO['predictedGender'].isin(acceptedGender)]


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraTotalName,extraProbability,extraCountryUsedByService,extraUsedAI,extraForcedGenderize
5106,11162,MI Jiujiang,,male,,CN,True,genderAPI.io,1,50,CN,False,False


In [14]:
result_Kaggle_First_Country_GenderAPIIO.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIIO/kaggleFirstCountry.csv", index=False)

The single mistake will be treated later. 

### GenderAPI.com

In [ ]:
result_Kaggle_First_Country_GenderAPIcom = await run_batched(
    wrapper_class=wrapper.GenderAPI_com_Wrapper,
    df=kaggle_First_Country,
    batch_size=50,
    pause_seconds=5,
    useFullName=False, 
    useLocalization=True
)
result_Kaggle_First_Country_GenderAPIcom


Processing rows 0-50...
Sleeping 7.2s to respect API limits...

Processing rows 50-100...
Sleeping 9.6s to respect API limits...

Processing rows 100-150...
Sleeping 6.1s to respect API limits...

Processing rows 150-200...
Sleeping 7.8s to respect API limits...

Processing rows 200-250...
Sleeping 6.5s to respect API limits...

Processing rows 250-300...
Sleeping 9.8s to respect API limits...

Processing rows 300-350...
Sleeping 6.4s to respect API limits...

Processing rows 350-400...
Sleeping 8.1s to respect API limits...

Processing rows 400-450...
Sleeping 8.9s to respect API limits...

Processing rows 450-500...
Sleeping 7.3s to respect API limits...

Processing rows 500-550...
Sleeping 8.0s to respect API limits...

Processing rows 550-600...
Sleeping 9.5s to respect API limits...

Processing rows 600-650...
Sleeping 9.5s to respect API limits...

Processing rows 650-700...
Sleeping 8.6s to respect API limits...

Processing rows 700-750...
Sleeping 7.1s to respect API limits...

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,4874,ALEKSANYAN Artur,Artur,male,male,AM,True,genderAPI.com,False,0.99,Artur,None
1,4875,AMOYAN Malkhas,Malkhas,male,male,AM,True,genderAPI.com,False,1.0,Malkhas,None
2,4876,GALSTYAN Slavik,Slavik,male,male,AM,True,genderAPI.com,False,0.99,Slavik,None
3,4877,HARUTYUNYAN Arsen,Arsen,male,male,AM,True,genderAPI.com,False,1.0,Arsen,None
4,4878,TEVANYAN Vazgen,Vazgen,male,male,AM,True,genderAPI.com,False,1.0,Vazgen,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9073,15980,HOMAN Khrystyna,Khrystyna,female,female,UA,True,genderAPI.com,False,1.0,Khrystyna,None
9074,15983,LIUZZI Emanuela,Emanuela,female,female,IT,True,genderAPI.com,False,0.98,Emanuela,None
9075,15984,BOERS Isayah,Isayah,male,male,NL,True,genderAPI.com,False,0.92,Isayah,None
9076,15985,STAUT Kevin,Kevin,male,male,FR,True,genderAPI.com,False,0.99,Kevin,None


In [9]:
result_Kaggle_First_Country_GenderAPIcom['predictedGender'].value_counts()

predictedGender
male       4713
female     4101
unknown      46
ERROR         7
Name: count, dtype: int64

In [10]:
result_Kaggle_First_Country_GenderAPIcom.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIcom/kaggleFirstCountry.csv", index=False)

As decided earlier, small quantities of mistake will be treated later to free up some strain on the budget, which is turning out to be more constrained than initially thought. 

### GenderGuesser

In [33]:
genderGuesserHandler = GenderGuesserHandler(kaggle_First_Country)

result_kaggle_First_GenderGuesser = genderGuesserHandler.get_prediction(useLocalization=True)

In [34]:
result_kaggle_First_GenderGuesser['extraPreciseGenderPredicted'].value_counts()

extraPreciseGenderPredicted
andy             3139
unknown          2071
male             1878
female           1760
mostly_female     118
mostly_male       112
Name: count, dtype: int64

In [35]:
result_kaggle_First_GenderGuesser.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderGuesser/kaggleFirstCountry.csv", index=False)

### Namsor

In [4]:
result_Kaggle_First_Country_NamSor = await run_batched(
    wrapper_class=wrapper.NamSorWrapper,
    df=kaggle_First_Country,
    batch_size=50,
    pause_seconds=2,
    endpoint=NamSorEndpoint.FIRST_NAME_GEO
)

Output()

[11:27:49] Processing rows 0-50...                                                                ]8;id=447680;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=832310;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[11:27:50] Cooling down for 3.5s to respect API limits...                                         ]8;id=379533;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=569048;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[11:27:54] Processing rows 50-100...                                                              ]8;id=654875;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=351050;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

           Cooling down for 5.6s to respect API limits...                                         ]8;id=495105;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=426004;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

...

[11:45:02] Cooling down for 3.5s to respect API limits...                                         ]8;id=494734;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=672773;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[11:45:05] Processing rows 9000-9050...                                                           ]8;id=226253;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=131171;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[11:45:06] Cooling down for 6.2s to respect API limits...                                         ]8;id=669233;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=249829;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[11:45:12] Processing rows 9050-9078...                                                           ]8;id=572289;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=703872;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

In [11]:
result_Kaggle_First_Country_NamSor.to_csv(
    "C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Namsor/kaggleFirstCountry.csv", 
    index=False
    )

## Wikidata First name NO COUNTRY

In [15]:
wikidata_First_NoCountry = master_df[
    (master_df['source'] == 'wikidata') &
    (master_df['firstName'].notnull()) 
]
wikidata_First_NoCountry.shape

(3937, 11)

### Genderize

In [8]:
result_Wikidata_First_NoCountry_Genderize = await run_batched(
    wrapper_class=wrapper.GenderizeWrapper,
    df=wikidata_First_NoCountry,
    batch_size=200,
    pause_seconds=0,
    useLocalization=False
)
print(result_Wikidata_First_NoCountry_Genderize.shape)
result_Wikidata_First_NoCountry_Genderize

Output()

[13:54:42] Processing rows 0-200...                                                               ]8;id=470418;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=23890;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[13:54:46] Cooling down for 1.9s to respect API limits...                                         ]8;id=649108;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=891353;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[13:54:48] Processing rows 200-400...                                                             ]8;id=66556;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=52273;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[13:54:52] Cooling down for 1.7s to respect API limits...                                         ]8;id=578937;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=941669;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

...

[13:56:59] Cooling down for 2.5s to respect API limits...                                         ]8;id=830710;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=863736;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[13:57:01] Processing rows 3800-3937...                                                           ]8;id=90224;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=403793;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

(3937, 10)


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,0,Clara Benson,Clara,female,female,GH,False,genderize.IO,247083,0.99
1,2,Dalia Ziada,Dalia,female,female,EG,False,genderize.IO,87214,0.99
2,3,Fatou Haidara,Fatou,female,female,ML,False,genderize.IO,25477,0.98
3,4,Claude Haffner,Claude,female,male,NaN,False,genderize.IO,161269,0.93
4,7,Diallo Sène,Diallo,female,male,ML,False,genderize.IO,8032,0.82
...,...,...,...,...,...,...,...,...,...,...
3932,4864,Ícaro Lira,Ícaro,male,male,BR,False,genderize.IO,324,1.00
3933,4865,Sir Chandler,Sir,male,male,AR,False,genderize.IO,3124,0.96
3934,4866,Julio Fierro,Julio,male,male,CL,False,genderize.IO,478589,1.00
3935,4870,Wrays Pérez,Wrays,male,male,PE,False,genderize.IO,3,1.00


In [23]:
result_Wikidata_First_NoCountry_Genderize['predictedGender'].value_counts(dropna=False)

predictedGender
male      2004
female    1887
None        46
Name: count, dtype: int64

In [22]:
result_Wikidata_First_NoCountry_Genderize[
    ~result_Wikidata_First_NoCountry_Genderize['predictedGender'].isin(['female', 'male'])
    ]

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
76,116,Charlbi Dean,Charlbi,female,None,ZA,False,genderize.IO,0,0.0
180,246,Uyaiedu Ikpe-Etim,Uyaiedu,female,None,NG,False,genderize.IO,0,0.0
295,386,Anastasiya-Alexandra Nenova,Anastasiya-Alexandra,female,None,ZA,False,genderize.IO,0,0.0
325,434,Ms Cosmo,Ms,female,None,ZA,False,genderize.IO,0,0.0
357,479,Eldevina Materula,Eldevina,female,None,MZ,False,genderize.IO,0,0.0
427,560,Mimlu Sen,Mimlu,female,None,IN,False,genderize.IO,0,0.0
619,781,Valancina Hiruć-Rusakievič,Valancina,female,None,NaN,False,genderize.IO,0,0.0
644,808,Otokata Higashi,Otokata,female,None,JP,False,genderize.IO,0,0.0
672,842,Rakuko Naito,Rakuko,female,None,JP,False,genderize.IO,0,0.0
735,916,Olyria Roy,Olyria,female,None,NaN,False,genderize.IO,0,0.0


In [26]:
result_Wikidata_First_NoCountry_Genderize.loc[
    ~result_Wikidata_First_NoCountry_Genderize['predictedGender'].isin(['female', 'male']), 'predictedGender'
] = 'unknown'

In [27]:
print(result_Wikidata_First_NoCountry_Genderize['predictedGender'].value_counts(dropna=False))
result_Wikidata_First_NoCountry_Genderize['predictedGender'].value_counts(dropna=False).sum()

predictedGender
male       2004
female     1887
unknown      46
Name: count, dtype: int64


np.int64(3937)

In [29]:
result_Wikidata_First_NoCountry_Genderize.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Genderize/wikidataFirstNoCountry.csv", index=False)

It appears that some names were not within the Genderize's database, and thus were unable to be given a predicted gender. It appears clearly that the tool didn't fail per se, it still answered in full capacity with the extra count and probability as requested, but failed to predict a gender. 

This wasn't quite catched in the first round of testing on the first dataset (Wikidata/Kaggle with first and country). **A step of full preprocessing of the testing datasets themselves will be in order.**

To myself : This should have been caught in the wrapper itself. 

### GenderAPI.io

In [36]:
result_Wikidata_First_NoCountry_GenderAPIIO = await run_batched(
    wrapper_class=wrapper.GenderAPI_IO_Wrapper,
    df=wikidata_First_NoCountry,
    batch_size=200,
    pause_seconds=0,
    useLocalization=False
)
print(result_Wikidata_First_NoCountry_GenderAPIIO.shape)
result_Wikidata_First_NoCountry_GenderAPIIO

Output()

[15:44:01] Processing rows 0-200...                                                               ]8;id=184489;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=974870;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:44:03] Cooling down for 2.1s to respect API limits...                                         ]8;id=398675;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=666578;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:44:05] Processing rows 200-400...                                                             ]8;id=960700;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=590328;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:44:07] Cooling down for 2.0s to respect API limits...                                         ]8;id=635527;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=160625;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

...

[15:45:27] Cooling down for 2.5s to respect API limits...                                         ]8;id=536567;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=741273;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:45:29] Processing rows 3800-3937...                                                           ]8;id=203654;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=390084;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

(3937, 13)


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraTotalName,extraProbability,extraCountryUsedByService,extraUsedAI,extraForcedGenderize
0,0,Clara Benson,clara,female,female,GH,False,genderAPI.io,9346,99,BR,False,False
1,2,Dalia Ziada,dalia,female,female,EG,False,genderAPI.io,2134,100,MX,False,False
2,3,Fatou Haidara,fatou,female,female,ML,False,genderAPI.io,121,100,US,False,False
3,4,Claude Haffner,claude,female,male,NaN,False,genderAPI.io,1561,91,US,False,False
4,7,Diallo Sène,diallo,female,male,ML,False,genderAPI.io,79,90,US,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3932,4864,Ícaro Lira,ícaro,male,male,BR,False,genderAPI.io,532,100,BR,False,False
3933,4865,Sir Chandler,sir,male,male,AR,False,genderAPI.io,792,97,US,False,False
3934,4866,Julio Fierro,julio,male,male,CL,False,genderAPI.io,24561,100,BR,False,False
3935,4870,Wrays Pérez,,male,null,PE,False,genderAPI.io,1,50,,False,False


In [38]:
result_Wikidata_First_NoCountry_GenderAPIIO

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraTotalName,extraProbability,extraCountryUsedByService,extraUsedAI,extraForcedGenderize
0,0,Clara Benson,clara,female,female,GH,False,genderAPI.io,9346,99,BR,False,False
1,2,Dalia Ziada,dalia,female,female,EG,False,genderAPI.io,2134,100,MX,False,False
2,3,Fatou Haidara,fatou,female,female,ML,False,genderAPI.io,121,100,US,False,False
3,4,Claude Haffner,claude,female,male,NaN,False,genderAPI.io,1561,91,US,False,False
4,7,Diallo Sène,diallo,female,male,ML,False,genderAPI.io,79,90,US,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3932,4864,Ícaro Lira,ícaro,male,male,BR,False,genderAPI.io,532,100,BR,False,False
3933,4865,Sir Chandler,sir,male,male,AR,False,genderAPI.io,792,97,US,False,False
3934,4866,Julio Fierro,julio,male,male,CL,False,genderAPI.io,24561,100,BR,False,False
3935,4870,Wrays Pérez,,male,null,PE,False,genderAPI.io,1,50,,False,False


In [40]:
result_Wikidata_First_NoCountry_GenderAPIIO['predictedGender'].value_counts(dropna=False)

predictedGender
male              1977
female            1878
null                81
gender unknown       1
Name: count, dtype: int64

In [42]:
result_Wikidata_First_NoCountry_GenderAPIIO.loc[
    ~result_Wikidata_First_NoCountry_GenderAPIIO['predictedGender'].isin(['female', 'male']) , 
    'predictedGender'
] = 'unkwnown'
result_Wikidata_First_NoCountry_GenderAPIIO['predictedGender'].value_counts(dropna=False)

predictedGender
male        1977
female      1878
unkwnown      82
Name: count, dtype: int64

In [43]:
result_Wikidata_First_NoCountry_GenderAPIIO.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIIO/wikidataFirstNoCountry.csv", index=False)

### GenderAPI.com

In [5]:
result_Wikidata_First_NoCountry_GenderAPIcom = await run_batched(
    wrapper_class=wrapper.GenderAPI_com_Wrapper,
    df=wikidata_First_NoCountry,
    batch_size=50,
    pause_seconds=1,
    useFullName=False, 
    useLocalization=False
)
print(result_Wikidata_First_NoCountry_GenderAPIcom.shape)
result_Wikidata_First_NoCountry_GenderAPIcom.head()

Output()

[14:10:25] Processing rows 0-50...                                                                ]8;id=76753;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=150342;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

           Cooling down for 2.1s to respect API limits...                                         ]8;id=456787;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=301385;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[14:10:27] Processing rows 50-100...                                                              ]8;id=399034;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=829263;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[14:10:28] Cooling down for 4.3s to respect API limits...                                         ]8;id=132446;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=125081;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

...

           Cooling down for 5.7s to respect API limits...                                         ]8;id=507540;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=638923;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[14:16:27] Processing rows 3900-3937...                                                           ]8;id=732628;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=890179;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

(3937, 12)


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,0,Clara Benson,Clara,female,female,GH,False,genderAPI.com,False,0.97,Clara,None
1,2,Dalia Ziada,Dalia,female,female,EG,False,genderAPI.com,False,0.99,Dalia,None
2,3,Fatou Haidara,Fatou,female,female,ML,False,genderAPI.com,False,0.97,Fatou,None
3,4,Claude Haffner,Claude,female,male,NaN,False,genderAPI.com,False,0.89,Claude,None
4,7,Diallo Sène,Diallo,female,male,ML,False,genderAPI.com,False,0.81,Diallo,None


In [7]:
print(result_Wikidata_First_NoCountry_GenderAPIcom.info())
result_Wikidata_First_NoCountry_GenderAPIcom

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3937 entries, 0 to 3936
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   index                     3937 non-null   int64  
 1   fullName                  3937 non-null   object 
 2   namePassed                3937 non-null   object 
 3   correctGender             3937 non-null   object 
 4   predictedGender           3840 non-null   object 
 5   localization              3645 non-null   object 
 6   useLocalization           3937 non-null   bool   
 7   serviceUsed               3937 non-null   object 
 8   extra_useFullName         3937 non-null   bool   
 9   extraProbability          3840 non-null   float64
 10  extraIdentifiedFirstName  3840 non-null   object 
 11  extraIdentifiedLastName   0 non-null      object 
dtypes: bool(2), float64(1), int64(1), object(8)
memory usage: 315.4+ KB
None


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
0,0,Clara Benson,Clara,female,female,GH,False,genderAPI.com,False,0.97,Clara,None
1,2,Dalia Ziada,Dalia,female,female,EG,False,genderAPI.com,False,0.99,Dalia,None
2,3,Fatou Haidara,Fatou,female,female,ML,False,genderAPI.com,False,0.97,Fatou,None
3,4,Claude Haffner,Claude,female,male,NaN,False,genderAPI.com,False,0.89,Claude,None
4,7,Diallo Sène,Diallo,female,male,ML,False,genderAPI.com,False,0.81,Diallo,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3932,4864,Ícaro Lira,Ícaro,male,male,BR,False,genderAPI.com,False,0.99,Ícaro,None
3933,4865,Sir Chandler,Sir,male,male,AR,False,genderAPI.com,False,0.97,Sir,None
3934,4866,Julio Fierro,Julio,male,male,CL,False,genderAPI.com,False,0.96,Julio,None
3935,4870,Wrays Pérez,Wrays,male,male,PE,False,genderAPI.com,False,0.75,Wrays,None


In [8]:
result_Wikidata_First_NoCountry_GenderAPIcom['predictedGender'].value_counts(dropna=False)

predictedGender
male       1984
female     1832
None         97
unknown      24
Name: count, dtype: int64

In [9]:
result_Wikidata_First_NoCountry_GenderAPIcom[result_Wikidata_First_NoCountry_GenderAPIcom['predictedGender'].isna()]

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extra_useFullName,extraProbability,extraIdentifiedFirstName,extraIdentifiedLastName
278,366,Ndeutala Angolo,Ndeutala,female,None,NaN,False,genderAPI.com,False,NaN,None,None
302,394,Manamba Kanté,Manamba,female,None,GN,False,genderAPI.com,False,NaN,None,None
315,411,Zaharouna Haoudadji,Zaharouna,female,None,KM,False,genderAPI.com,False,NaN,None,None
336,452,Ergoge Tesfaye,Ergoge,female,None,ET,False,genderAPI.com,False,NaN,None,None
357,479,Eldevina Materula,Eldevina,female,None,MZ,False,genderAPI.com,False,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3583,4366,Bullumba Landestoy,Bullumba,male,None,DO,False,genderAPI.com,False,NaN,None,None
3711,4541,Zaronn Cann,Zaronn,male,None,CL,False,genderAPI.com,False,NaN,None,None
3778,4638,Trelonnie Owens,Trelonnie,male,None,UY,False,genderAPI.com,False,NaN,None,None
3826,4696,J Chaiane,J,male,None,UY,False,genderAPI.com,False,NaN,None,None


Upon further testing with the API directly from the website itself, it seems pretty fair to assume that those NA result heres are equivalents to unknowns. Using Insomnia to test out individual samples from this list that result in NA, the JSON we get in answer is different, and simply say "result_found": "false". 

It is my opinion that thsoe can be thus classified as unknown, since the tool fails to produce a classification of any kind.

In [16]:
result_Wikidata_First_NoCountry_GenderAPIcom['predictedGender'].fillna('unknown', inplace=True)

In [17]:
result_Wikidata_First_NoCountry_GenderAPIcom['predictedGender'].value_counts(dropna=False)

predictedGender
male       1984
female     1832
unknown     121
Name: count, dtype: int64

In [18]:
result_Wikidata_First_NoCountry_GenderAPIcom.to_csv("C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIcom/wikidataFirstNoCountry.csv", index=False)

### Namsor

In [4]:
result_Wikidata_first_NoCountry_Namsor = await run_batched(
    wrapper_class=wrapper.NamSorWrapper,
    df=wikidata_First_NoCountry,
    batch_size=50,
    pause_seconds=3,
    endpoint=NamSorEndpoint.FIRST_NAME
)

Output()

[15:17:00] Processing rows 0-50...                                                                ]8;id=886112;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=949815;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:17:01] Cooling down for 6.7s to respect API limits...                                         ]8;id=615860;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=300713;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:17:08] Processing rows 50-100...                                                              ]8;id=49801;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=989301;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:17:09] Cooling down for 7.3s to respect API limits...                                         ]8;id=531122;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=496047;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

...

[15:26:35] Cooling down for 6.0s to respect API limits...                                         ]8;id=118120;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=645528;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:26:42] Processing rows 3850-3900...                                                           ]8;id=976535;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=561172;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:26:43] Cooling down for 6.3s to respect API limits...                                         ]8;id=715832;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=519839;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:26:49] Processing rows 3900-3937...                                                           ]8;id=178384;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=792644;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

In [5]:
print(result_Wikidata_first_NoCountry_Namsor.shape)
print(result_Wikidata_first_NoCountry_Namsor.info())
result_Wikidata_first_NoCountry_Namsor

(3937, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3937 entries, 0 to 3936
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       3937 non-null   int64  
 1   fullName                    3937 non-null   object 
 2   namePassed                  3937 non-null   object 
 3   correctGender               3937 non-null   object 
 4   predictedGender             3937 non-null   object 
 5   localization                3645 non-null   object 
 6   useLocalization             3937 non-null   bool   
 7   serviceUsed                 3937 non-null   object 
 8   extraGenderScale            3937 non-null   float64
 9   extraScore                  3937 non-null   float64
 10  extraProbabilityCalibrated  3937 non-null   float64
 11  extraScript                 3937 non-null   object 
dtypes: bool(1), float64(3), int64(1), object(7)
memory usage: 342.3+ KB
None


,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraGenderScale,extraScore,extraProbabilityCalibrated,extraScript
0,0,Clara Benson,Clara,female,female,GH,False,NamSor,0.984368,12.093966,0.992184,LATIN
1,2,Dalia Ziada,Dalia,female,female,EG,False,NamSor,0.986410,12.673898,0.993205,LATIN
2,3,Fatou Haidara,Fatou,female,female,ML,False,NamSor,0.963018,8.663081,0.981509,LATIN
3,4,Claude Haffner,Claude,female,male,NaN,False,NamSor,-0.551143,4.129962,0.775572,LATIN
4,7,Diallo Sène,Diallo,female,male,ML,False,NamSor,-0.868756,6.984416,0.934378,LATIN
...,...,...,...,...,...,...,...,...,...,...,...,...
3932,4864,Ícaro Lira,Ícaro,male,male,BR,False,NamSor,-0.955574,8.416326,0.977787,LATIN
3933,4865,Sir Chandler,Sir,male,male,AR,False,NamSor,-0.947264,8.125483,0.973632,LATIN
3934,4866,Julio Fierro,Julio,male,male,CL,False,NamSor,-0.971003,10.618185,0.985502,LATIN
3935,4870,Wrays Pérez,Wrays,male,male,PE,False,NamSor,-0.185137,1.166812,0.592569,LATIN


In [11]:
result_Wikidata_first_NoCountry_Namsor['predictedGender'].value_counts(dropna=False)

predictedGender
male      2018
female    1919
Name: count, dtype: int64

In [12]:
result_Wikidata_first_NoCountry_Namsor.to_csv(
    "C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Namsor/wikidataFirstNoCountry.csv",
    index=False
)

### Gender Guesser

In [36]:
genderGuesserHandler = GenderGuesserHandler(wikidata_First_NoCountry)

result_Wikidata_First_NoCountry_GenderGuesser = genderGuesserHandler.get_prediction(useLocalization=False)

In [37]:
result_Wikidata_First_NoCountry_GenderGuesser['extraPreciseGenderPredicted'].value_counts()

extraPreciseGenderPredicted
male             1417
female           1413
unknown           880
andy               93
mostly_male        76
mostly_female      58
Name: count, dtype: int64

In [38]:
result_Wikidata_First_NoCountry_GenderGuesser.to_csv(
    "C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderGuesser/wikidataFirstNoCountry.csv", 
    index=False)

## Kaggle First + No Country

In [7]:
master_df['source'].value_counts()

source
kaggleOlympic    11113
wikidata          4874
Name: count, dtype: int64

In [4]:
kaggle_First_NoCountry = master_df[
    (master_df['source'] == 'kaggleOlympic') &
    (master_df['firstName'].notnull()) 
]
kaggle_First_NoCountry.shape

(9082, 11)

### Gender Guesser

In [39]:
genderGuesserHandler = GenderGuesserHandler(kaggle_First_NoCountry)

result_Kaggle_First_NoCountry_GenderGuesser = genderGuesserHandler.get_prediction(useLocalization=False)

In [40]:
result_Kaggle_First_NoCountry_GenderGuesser['extraPreciseGenderPredicted'].value_counts(dropna=False)

extraPreciseGenderPredicted
male             3374
female           3135
unknown          2073
mostly_male       183
mostly_female     170
andy              147
Name: count, dtype: int64

In [41]:
result_Kaggle_First_NoCountry_GenderGuesser.to_csv(
    "C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderGuesser/kaggleFirstNoCountry.csv", 
    index=False)

### Genderize

In [25]:
result_kaggle_First_NoCountry_Genderize = await run_batched(
    wrapper_class=wrapper.GenderizeWrapper,
    df=kaggle_First_NoCountry,
    batch_size=50,
    pause_seconds=5,
    useLocalization=False
)

Output()

[14:27:16] Processing rows 0-50...                                                                ]8;id=159723;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=453938;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[14:27:32] Cooling down for 6.7s to respect API limits...                                         ]8;id=426937;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=610348;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[14:27:39] Processing rows 50-100...                                                              ]8;id=527925;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=50528;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[14:27:43] Cooling down for 8.3s to respect API limits...                                         ]8;id=700786;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=777004;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

...

[15:01:46] Cooling down for 7.6s to respect API limits...                                         ]8;id=226306;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=709326;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:01:53] Processing rows 9000-9050...                                                           ]8;id=621646;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=496869;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:01:57] Cooling down for 6.6s to respect API limits...                                         ]8;id=469438;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=989511;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:02:04] Processing rows 9050-9082...                                                           ]8;id=365624;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=471950;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

In [26]:
result_kaggle_First_NoCountry_Genderize.shape

(9082, 10)

In [27]:
result_kaggle_First_NoCountry_Genderize.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9082 entries, 0 to 9081
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             9082 non-null   int64  
 1   fullName          9082 non-null   object 
 2   namePassed        9082 non-null   object 
 3   correctGender     9082 non-null   object 
 4   predictedGender   8971 non-null   object 
 5   localization      9078 non-null   object 
 6   useLocalization   9082 non-null   bool   
 7   serviceUsed       9082 non-null   object 
 8   extraCount        9082 non-null   int64  
 9   extraProbability  9082 non-null   float64
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 647.6+ KB


In [28]:
result_kaggle_First_NoCountry_Genderize[result_kaggle_First_NoCountry_Genderize['predictedGender'].isna()]

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
45,4947,MAGOMEDOMAROV Magomedomar,Magomedomar,male,None,RU,False,genderize.IO,0,0.0
47,4949,BISHRELT Khorloodoi,Khorloodoi,female,None,MN,False,genderize.IO,0,0.0
142,5094,TSEGAY Gudaf,Gudaf,female,None,ET,False,genderize.IO,0,0.0
203,5177,GERBEKOV Askerbii,Askerbii,male,None,RU,False,genderize.IO,0,0.0
218,5195,CAMERON Lloydricia,Lloydricia,female,None,US,False,genderize.IO,0,0.0
...,...,...,...,...,...,...,...,...,...,...
8926,15791,JOSAIA Raisuqe,Raisuqe,male,None,FJ,False,genderize.IO,0,0.0
8941,15810,RAVUTAUMADA Selesitino,Selesitino,male,None,FJ,False,genderize.IO,0,0.0
8945,15814,APHIBARNRAT Kiradech,Kiradech,male,None,TH,False,genderize.IO,0,0.0
8955,15827,LALOMILO Lalomilo,Lalomilo,male,None,WS,False,genderize.IO,0,0.0


In [29]:
result_kaggle_First_NoCountry_Genderize.to_csv(
    "C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/Genderize/kaggleFirstNoCountry.csv",
    index=False
)

### GenderAPI.io

In [30]:
result_Kaggle_First_NoCountry_GenderAPIIO = await run_batched(
    wrapper_class=wrapper.GenderAPI_IO_Wrapper,
    df=kaggle_First_NoCountry,
    batch_size=50,
    pause_seconds=3,
    useLocalization=False
)

Output()

[15:22:43] Processing rows 0-50...                                                                ]8;id=771975;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=600388;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:22:53] Processing rows 50-100...                                                              ]8;id=218893;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=701718;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:23:08] Cooling down for 5.1s to respect API limits...                                         ]8;id=721963;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=236200;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:23:13] Processing rows 100-150...                                                             ]8;id=152553;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=903224;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

...

[15:47:13] Cooling down for 5.0s to respect API limits...                                         ]8;id=607919;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=67946;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:47:18] Processing rows 9000-9050...                                                           ]8;id=406679;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=471765;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

[15:47:20] Cooling down for 7.9s to respect API limits...                                         ]8;id=661876;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=955679;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#61\61]8;;\

[15:47:28] Processing rows 9050-9082...                                                           ]8;id=423120;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py\run_batched.py]8;;\:]8;id=321257;file://c:\Users\brief\Desktop\Master\Ecole\Semestre3\M8_ProjetDeRecherche\code_ProjetDeRecherche\notebooks\helpers\run_batched.py#48\48]8;;\

All batches completed successfully!

In [31]:
result_Kaggle_First_NoCountry_GenderAPIIO.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9082 entries, 0 to 9081
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   index                      9082 non-null   int64 
 1   fullName                   9082 non-null   object
 2   namePassed                 9081 non-null   object
 3   correctGender              9082 non-null   object
 4   predictedGender            9082 non-null   object
 5   localization               9078 non-null   object
 6   useLocalization            9082 non-null   bool  
 7   serviceUsed                9082 non-null   object
 8   extraTotalName             9082 non-null   int64 
 9   extraProbability           9082 non-null   int64 
 10  extraCountryUsedByService  9081 non-null   object
 11  extraUsedAI                9082 non-null   bool  
 12  extraForcedGenderize       9082 non-null   bool  
dtypes: bool(3), int64(3), object(7)
memory usage: 736.3+ KB


In [36]:
result_Kaggle_First_NoCountry_GenderAPIIO.loc[
    result_Kaggle_First_NoCountry_GenderAPIIO['predictedGender']=='null', 'predictedGender'
] = 'unknown'

In [39]:
result_Kaggle_First_NoCountry_GenderAPIIO['fullName'].value_counts(dropna=False)

fullName
WATANABE Yuta         2
FERNANDEZ Federico    2
HAMZA Malak           2
PEREZ Maria           2
ELSAYED Mohamed       2
                     ..
YAMAMOTO Shoma        1
TSUTSUE Kaito         1
GLENN Rachel          1
YOUNG Nico            1
BUTLER Aaliyah        1
Name: count, Length: 9072, dtype: int64

In [40]:
result_Kaggle_First_NoCountry_GenderAPIIO.to_csv(
    "C:/Users/brief/Desktop/Master/Ecole/Semestre3/M8_ProjetDeRecherche/code_ProjetDeRecherche/data/TestedData/GenderAPIIO/kaggleFirstNoCountry.csv",
    index=False
)

### GenderAPI.com

### NamSor